In [ ]:
import numpy as np

### Bell states with Numpy

In [ ]:
# One-qubit basis
def qubit_0():
    return np.array([1, 0], dtype=np.complex128)
def qubit_1():
    return np.array([0, 1], dtype=np.complex128)

# Two-qubit basis
def qubit_00():
    return np.kron(qubit_0(), qubit_0())
def qubit_01():
    return np.kron(qubit_0(), qubit_1())
def qubit_10():
    return np.kron(qubit_1(), qubit_0())
def qubit_11():
    return np.kron(qubit_1(), qubit_1())

# One-qubit gates Identity and Hadamard
I = np.array([[1, 0], [0, 1]])
H = (1/np.sqrt(2))*np.array([[1, 1], [1, -1]])

# Two-qubit gates CNOT and Hadamard
CNOT= np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
H1 = np.kron(H, I)
H2 = np.kron(I, H)

In [ ]:
# Bell states
def bell_phi_plus():
    #Apply Hadamard and CNOT on |00>
    return CNOT@H1@qubit_00()
def bell_phi_minus():
    #Apply Hadamard and CNOT on |10>
    return CNOT@H1@qubit_10()
def bell_psi_plus():
    #Apply Hadamard and CNOT on |01>
    return CNOT@H1@qubit_01()
def bell_psi_minus():
    #Apply Hadamard and CNOT on |11>
    return CNOT@H1@qubit_11()

### Bell states with qiskit

In [ ]:
import qiskit as qk
n_qubits = 2
n_cbits = 2

def create_qc(n_q, n_c):
    qreg = qk.QuantumRegister(n_q)
    creg = qk.ClassicalRegister(n_c)
    return qk.QuantumCircuit(qreg,creg)

def qk_bell_phi_plus():
    qc = create_qc(n_qubits, n_cbits)
    #Apply Hadamard and CNOT on |00> (default)
    qc.h(0)
    qc.cx(0,1)
    return qc
def qk_bell_phi_minus():
    qc = create_qc(n_qubits, n_cbits)
    #Apply Hadamard and CNOT on |10>
    qc.x(0) # Flip first qubit, so we have |10>
    qc.h(0)
    qc.cx(0,1)
    return qc
def qk_bell_psi_plus():
    qc = create_qc(n_qubits, n_cbits)
    #Apply Hadamard and CNOT on |01>
    qc.x(1) # Flip second qubit, so we have |01>
    qc.h(0)
    qc.cx(0, 1)
    return qc
def qk_bell_psi_minus():
    qc = create_qc(n_qubits, n_cbits)
    #Apply Hadamard and CNOT on |11>
    qc.x(qc.qubits) # Flip both qubits, so we have |11>
    qc.h(0)
    qc.cx(0,1)
    return qc

### Measurements with numpy

In [ ]:
from collections import Counter
def measure(state, n_shots=1):
    probabilities = np.abs(state**2)
    outcomes = np.arange(len(state))
    # Perform ’measurement’ n_shots times
    measured_outcomes = np.random.choice(outcomes, p=probabilities, size = n_shots)
    # Organise the results in a dict
    output = {}
    counts = Counter(measured_outcomes) # Aggregate outcomes in Counter object
    n_qubits = int(np.log2(len(state))) # Number of qubits to get the right binary format
    for k in counts.keys():
        output[format(k, f"0{n_qubits}b")] = counts[k]
    return output


### Measurements with qiskit

In [ ]:
import qiskit_aer
def qk_measure(qc, n_shots=1):
    qc.measure(qc.qubits, qc.clbits) # Add measurement to all qubits in circuit
    backend = qiskit_aer.Aer.get_backend("qasm_simulator") # Initialize backend
    job = backend.run(qc,shots=n_shots)
    return job.result().get_counts(qc)

### Demo

In [ ]:
# Numpy
measure(bell_phi_plus(), 1000)

In [ ]:
# Qiskit
qk_measure(qk_bell_phi_plus(), 1000)

### Distinguish $|\Phi_+\rangle$ and $|\Phi_-\rangle$

In [ ]:
# Numpy
measure(H1@H2@bell_phi_plus(), 1000)

In [ ]:
measure(H1@H2@bell_phi_minus(), 1000)

In [ ]:
# Qiskit
s1 = qk_bell_phi_plus()
s1.h(s1.qubits)
qk_measure(s1, 1000)

In [ ]:
s2 = qk_bell_phi_minus()
s2.h(s2.qubits)
qk_measure(s2, 1000)